<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/ML-2025a/blob/main/PINN_Plates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [59]:
# Install dependencies for Colab
!pip install pytorch-lightning clearml

import math
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from clearml import Task
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass

In [60]:
#Enter your code here to implement Step 2 of the logging instruction as it is shown below
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
%env CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
env: CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I


# 🧠 Theoretical Introduction: Dimensionless PINNs for Parallel Plate Flow

We consider the steady, fully developed flow of an incompressible Newtonian fluid between two infinite parallel plates separated by distance \(h\).  
The velocity is unidirectional along \(x\), varying only in the transverse direction \(y\), so the governing equation reduces to:

$$
\mu \frac{d^2 u}{dy^2} = \frac{dp}{dx}
$$

where $\mu$ is dynamic viscosity, $\frac{dp}{dx}$ — constant pressure gradient (typically negative).

The **no-slip boundary conditions** are applied at both plates:
<!--  -->
$$
u\left(-\tfrac{h}{2}\right) = 0, \quad u\left(\tfrac{h}{2}\right) = 0
$$

---

## ✅ Nondimensionalization

We define dimensionless variables:

$$
Y = \frac{y}{h}, \qquad U = \frac{u}{U_\mathrm{ref}}, \qquad
\Pi = \frac{h^2}{\mu U_\mathrm{ref}} \left(\frac{dp}{dx}\right)
$$

Substituting into the PDE gives the **dimensionless Poisson equation**:

$$
\frac{d^2 U}{dY^2} = \Pi
$$

with **boundary conditions**:

$$
U(-\tfrac{1}{2}) = 0, \quad U(\tfrac{1}{2}) = 0
$$

The **analytic solution** in nondimensional form is:

$$
U(Y) = \frac{\Pi}{2} \left( Y^2 - \tfrac{1}{4} \right)
$$

---

## ⚙️ Physics-Informed Neural Network (PINN) Formulation

We approximate $U(Y)$ using a neural network $\hat{U}(Y)$ and enforce physics via loss minimization.

The total loss combines:

- **PDE residual loss** — enforcing the governing equation  
- **Boundary loss** — enforcing no-slip at $Y = \pm \tfrac{1}{2}$

$$
\mathcal{L} =
\underbrace{
\frac{1}{N_r} \sum_{i=1}^{N_r}
\left|
\frac{d^2 \hat{U}}{dY^2}(Y_i) - \Pi
\right|^2
}_{\text{Physics loss}}
+
\underbrace{
\frac{1}{N_b} \sum_{j=1}^{N_b}
\left|
\hat{U}(Y_j) - U_j
\right|^2
}_{\text{Boundary loss}}
$$

# Config

In [61]:
torch.set_default_dtype(torch.float32)
pl.seed_everything(42)

# ----------------------------
# Config
# ----------------------------
@dataclass
class Config:
    # Physics parameters for dimensionalization
    h: float = 1e-3                # channel height [m]
    mu: float = 1e-6               # dynamic viscosity [Pa·s]
    rho: float = 1e+3              # density [kg / m^3]
    dpdx: float = -1e+3        # pressure gradient [Pa/m]

    # training hyperparams
    lr: float = 1e-3
    epochs: int = 1000
    n_colloc_per_Y: int = 64       # spatial points per batch
    n_boundary_per_Y: int = 2      # boundary points (fixed: Y=-0.5, 0.5)
    batch_size_pi: int = 20         # number of Pi values per batch
    n_pi_total: int = 100          # total number of Pi values
    pi_min: float = - 1e+3
    pi_max: float = - 1e+2

    # data split ratios
    train_ratio: float = 0.6
    val_ratio: float = 0.2
    test_ratio: float = 0.2

    # network
    layers: tuple = (2, 64, 64, 64, 64, 1)

    # settings
    check_val_every_n_epoch: int = 100
    task_name: str = "PINN_plates_dimless_generalized"

cfg = Config()

INFO:lightning_fabric.utilities.seed:Seed set to 42


# Dataset and Dataloaders

In [62]:
# ----------------------------
# Dataset Classes
# ----------------------------
class PiSpatialDataset(Dataset):
    """Dataset that returns batches of spatial points for specific Pi values"""
    def __init__(self, pi_values, n_colloc_per_Y, n_boundary_per_Y, device):
        self.pi_values = pi_values
        self.n_colloc_per_Y = n_colloc_per_Y
        self.n_boundary_per_Y = n_boundary_per_Y
        self.device = device

        # Precompute spatial grids
        self.Y_colloc = torch.linspace(-0.5, 0.5, n_colloc_per_Y, device=device).reshape(-1, 1)
        self.Y_boundary = torch.tensor([[-0.5], [0.5]], device=device, dtype=torch.float32)

    def __len__(self):
        return len(self.pi_values)

    def __getitem__(self, idx):
        pi = self.pi_values[idx]

        # Collocation points
        Y_colloc_batch = self.Y_colloc
        Pi_colloc_batch = torch.full_like(Y_colloc_batch, pi)
        X_colloc = torch.cat([Y_colloc_batch, Pi_colloc_batch], dim=1)

        # Boundary points
        Y_boundary_batch = self.Y_boundary
        Pi_boundary_batch = torch.full_like(Y_boundary_batch, pi)
        X_boundary = torch.cat([Y_boundary_batch, Pi_boundary_batch], dim=1)
        U_boundary = torch.zeros_like(Y_boundary_batch)

        return X_colloc, X_boundary, U_boundary

class PiBatchDataModule(pl.LightningDataModule):
    def __init__(self, cfg: Config, device="cpu"):
        super().__init__()
        self.cfg = cfg
        self.device = device

    def setup(self, stage=None):
        # Generate Pi values
        n_total = self.cfg.n_pi_total
        self.pi_all = torch.linspace(
            self.cfg.pi_min, self.cfg.pi_max, n_total, device=self.device
        )

        # Split into train/val/test
        n_train = int(n_total * self.cfg.train_ratio)
        n_val = int(n_total * self.cfg.val_ratio)
        n_test = n_total - n_train - n_val

        # Shuffle indices
        indices = torch.randperm(n_total)
        train_idx = indices[:n_train]
        val_idx = indices[n_train:n_train + n_val]
        test_idx = indices[n_train + n_val:]

        self.pi_train = self.pi_all[train_idx]
        self.pi_val = self.pi_all[val_idx]
        self.pi_test = self.pi_all[test_idx]

        print(f"Dataset sizes - Train: {len(self.pi_train)}, Val: {len(self.pi_val)}, Test: {len(self.pi_test)}")

    def train_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_train,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi,
                          num_workers=0, shuffle=True)

    def val_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_val,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi,
                          num_workers=0, shuffle=False)

    def test_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_test,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi, shuffle=False)

# Model

In [ ]:
task = Task.init(
    project_name="PINN_Project",   # choose a descriptive project name
    task_name=cfg.task_name,  # descriptive task name
)

# task.connect(params)
tb_logger = TensorBoardLogger(
    save_dir="logs",  # ClearML will automatically monitor this
    name=cfg.task_name,
    # version=f"bs_{HP['batch_size']}_lr_{HP['lr_simclr']}"
  )

# ----------------------------
# PINN Model
# ----------------------------
class PINN(pl.LightningModule):
    def __init__(self, layers, lr):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.net = self._build_mlp(layers)

    def _build_mlp(self, sizes):
        layers = []
        for i in range(len(sizes)-1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes)-2:
                layers.append(nn.Tanh())
        return nn.Sequential(*layers)

    def forward(self, X):
        return self.net(X)

    def _second_deriv_wrt_Y(self, X):
        X_temp = X.clone().requires_grad_(True)
        U = self.forward(X_temp)

        # First derivative wrt Y
        dU_dY = torch.autograd.grad(
            U, X_temp,
            grad_outputs=torch.ones_like(U),
            create_graph=True,
            retain_graph=True
        )[0][:, 0:1]

        # Second derivative wrt Y
        d2U_dY2 = torch.autograd.grad(
            dU_dY, X_temp,
            grad_outputs=torch.ones_like(dU_dY),
            create_graph=True,
            retain_graph=True
        )[0][:, 0:1]

        return d2U_dY2

    def training_step(self, batch, batch_idx):
        X_colloc, X_bc, U_bc = batch

        # Reshape batches: [batch_size_pi * n_points, 2]
        batch_size_pi = X_colloc.shape[0]
        n_colloc = X_colloc.shape[1]
        n_bc = X_bc.shape[1]

        X_colloc_flat = X_colloc.reshape(-1, 2)
        X_bc_flat = X_bc.reshape(-1, 2)
        U_bc_flat = U_bc.reshape(-1, 1)

        # PDE residual loss
        res = self._pde_residual(X_colloc_flat)
        pde_loss = torch.mean(res**2)
        # constraint = torch.mean(res)

        # Boundary loss
        U_pred_bc = self.forward(X_bc_flat)
        bc_loss = nn.functional.mse_loss(U_pred_bc, U_bc_flat)

        loss = pde_loss + bc_loss #+ constraint
        #  + 0.01 * physical_constraint_loss
        self.log_dict({
            "train_pde": pde_loss,
            "train_bc": bc_loss,
            # "train_pc": physical_constraint_loss,
            "train_loss": loss
        }, prog_bar=True)
        return loss

    def _pde_residual(self, X_colloc):
        U_YY = self._second_deriv_wrt_Y(X_colloc)
        Pi = X_colloc[:, 1:2]
        res = U_YY - Pi
        return res

    def validation_step(self, batch, batch_idx):
        X_colloc, X_bc, U_bc = batch

        # Flatten batches
        X_colloc_flat = X_colloc.reshape(-1, 2)
        X_bc_flat = X_bc.reshape(-1, 2)
        U_bc_flat = U_bc.reshape(-1, 1)

        with torch.enable_grad():
            res = self._pde_residual(X_colloc_flat)
            pde_loss = torch.mean(res ** 2)
            # constraint = torch.mean(res)
            U_pred_bc = self.forward(X_bc_flat)
            bc_loss = nn.functional.mse_loss(U_pred_bc, U_bc_flat)
            val_loss = pde_loss + bc_loss# + constraint

        self.log_dict({
            "val_pde": pde_loss,
            "val_bc": bc_loss,
            "val_loss": val_loss
        }, prog_bar=True)
        return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-5)

# ----------------------------
# Dimensionalization Utilities
# ----------------------------
def dimensionless_to_dimensional(U_dim, Y_dim, Pi, cfg):
    """Convert dimensionless results to dimensional"""
    # Velocity: u = U * U_ref
    u_dimensional = U_dim * cfg.U_ref

    # Position: y = Y * h
    y_dimensional = Y_dim * cfg.h

    # Pressure gradient recovery: dp/dx = (Π * μ * U_ref) / h²
    dpdx_dimensional = (Pi * cfg.mu * cfg.U_ref) / (cfg.h**2)

    return u_dimensional, y_dimensional, dpdx_dimensional

def analytical_solution_dimensional(y, dpdx, mu, h):
    """Analytical solution for dimensional velocity"""
    # u(y) = (1/(2μ)) * (dp/dx) * (y² - (h/2)²)
    return (1/(2*mu)) * dpdx * (y**2 - (h/2)**2)

# ----------------------------
# Training
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
dm = PiBatchDataModule(cfg, device=device)
dm.setup()

model = PINN(layers=cfg.layers, lr=cfg.lr)

ckpt_cb = ModelCheckpoint(
    dirpath="checkpoints",
    filename="pinn_best",
    monitor="val_loss",
    save_top_k=1,
    mode="min",
)

trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    accelerator="auto",
    callbacks=[ckpt_cb],
    log_every_n_steps=50,
    check_val_every_n_epoch=cfg.check_val_every_n_epoch
)

trainer.fit(model, datamodule=dm)


ClearML Task: created new task id=777f4a2ab87b405aba5fb62681c95ebf
ClearML results page: https://app.clear.ml/projects/3b8e0332a8dc4f6ba4a5abbb428a0880/experiments/777f4a2ab87b405aba5fb62681c95ebf/output/log


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: UserWarning:

Checkpoint directory /content/checkpoints exists and is not empty.

INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 12.7 K | train
--------------------------------------------
12.7 K    Trainable params
0         Non-trainable params
12.7 K    Total params
0.051     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Dataset sizes - Train: 60, Val: 20, Test: 20
Dataset sizes - Train: 60, Val: 20, Test: 20


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: |          | 0/? [00:00<?, ?it/s]

ClearML Task: created new task id=777f4a2ab87b405aba5fb62681c95ebf
ClearML results page: https://app.clear.ml/projects/3b8e0332a8dc4f6ba4a5abbb428a0880/experiments/777f4a2ab87b405aba5fb62681c95ebf/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# # ----------------------------
# # Testing and Results
# # ----------------------------
# best_model = PINN.load_from_checkpoint(ckpt_cb.best_model_path, layers=cfg.layers, lr=cfg.lr)
# best_model.eval()
# best_model = best_model.to(device)

# # Test on various Pi values
# test_pi_values = [1.0, 3.0, 5.0, 7.0, 10.0]
# Y_test = torch.linspace(-0.5, 0.5, 100, device=device).reshape(-1, 1)

# # Results tables
# dimensionless_results = []
# dimensional_results = []

# plt.figure(figsize=(15, 6))

# # Plot 1: Dimensionless results
# plt.subplot(1, 2, 1)
# for pi in test_pi_values:
#     Pi_test = torch.full_like(Y_test, pi)
#     X_test = torch.cat([Y_test, Pi_test], dim=1)

#     with torch.no_grad():
#         U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

#     U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

#     # Dimensionless errors
#     mse_dimless = np.mean((U_pred_dimless - U_analytical_dimless)**2)
#     max_error_dimless = np.max(np.abs(U_pred_dimless - U_analytical_dimless))

#     dimensionless_results.append({
#         "Pi": pi,
#         "MSE": mse_dimless,
#         "Max_Error": max_error_dimless,
#         "U_pred_center": U_pred_dimless[50],  # Y=0
#         "U_analytical_center": U_analytical_dimless[50],
#         "Error_center": abs(U_pred_dimless[50] - U_analytical_dimless[50])
#     })

#     plt.plot(Y_test.cpu().numpy(), U_pred_dimless, '--', linewidth=2, label=f'PINN Π={pi}')
#     plt.plot(Y_test.cpu().numpy(), U_analytical_dimless, '-', alpha=0.7, label=f'Analytical Π={pi}')

# plt.xlabel('Y (Dimensionless)')
# plt.ylabel('U (Dimensionless)')
# plt.title('Dimensionless Velocity Profiles')
# plt.legend()
# plt.grid(True)

# # Plot 2: Dimensional results
# plt.subplot(1, 2, 2)
# for pi in test_pi_values:
#     Pi_test = torch.full_like(Y_test, pi)
#     X_test = torch.cat([Y_test, Pi_test], dim=1)

#     with torch.no_grad():
#         U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

#     # Convert to dimensional
#     u_pred_dimensional, y_dimensional, dpdx_recovered = dimensionless_to_dimensional(
#         U_pred_dimless, Y_test.cpu().numpy().flatten(), pi, cfg
#     )

#     # Analytical solution in dimensional form
#     u_analytical_dimensional = analytical_solution_dimensional(
#         y_dimensional, dpdx_recovered, cfg.mu, cfg.h
#     )

#     # Dimensional errors
#     mse_dimensional = np.mean((u_pred_dimensional - u_analytical_dimensional)**2)
#     max_error_dimensional = np.max(np.abs(u_pred_dimensional - u_analytical_dimensional))

#     dimensional_results.append({
#         "Pi": pi,
#         "dpdx_recovered [Pa/m]": dpdx_recovered,
#         "MSE [m²/s²]": mse_dimensional,
#         "Max_Error [m/s]": max_error_dimensional,
#         "u_pred_center [m/s]": u_pred_dimensional[50],
#         "u_analytical_center [m/s]": u_analytical_dimensional[50],
#         "Error_center [m/s]": abs(u_pred_dimensional[50] - u_analytical_dimensional[50])
#     })

#     plt.plot(y_dimensional, u_pred_dimensional, '--', linewidth=2, label=f'PINN Π={pi}')
#     plt.plot(y_dimensional, u_analytical_dimensional, '-', alpha=0.7, label=f'Analytical Π={pi}')

# plt.xlabel('y [m]')
# plt.ylabel('u [m/s]')
# plt.title('Dimensional Velocity Profiles')
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()

# # Print results tables
# print("\n" + "="*80)
# print("DIMENSIONLESS RESULTS")
# print("="*80)
# df_dimensionless = pd.DataFrame(dimensionless_results)
# pd.set_option("display.float_format", "{:.6e}".format)
# print(df_dimensionless.to_string(index=False))

# print("\n" + "="*80)
# print("DIMENSIONAL RESULTS")
# print("="*80)
# df_dimensional = pd.DataFrame(dimensional_results)
# print(df_dimensional.to_string(index=False))

# # Summary statistics
# print("\n" + "="*80)
# print("SUMMARY STATISTICS")
# print("="*80)
# print(f"Training Pi range: {cfg.pi_min} to {cfg.pi_max}")
# print(f"Test Pi values: {test_pi_values}")
# print(f"\nDimensionless Max Error across all tests: {max(r['Max_Error'] for r in dimensionless_results):.2e}")
# print(f"Dimensional Max Error across all tests: {max(r['Max_Error [m/s]'] for r in dimensional_results):.2e} m/s")

# # Test on validation and test sets
# print("\n" + "="*80)
# print("VALIDATION AND TEST SET PERFORMANCE")
# print("="*80)

# def evaluate_dataset(pi_values, dataset_name):
#     total_loss = 0
#     with torch.no_grad():
#         for pi in pi_values[:5]:  # Evaluate on first 5 values for brevity
#             Pi_test = torch.full_like(Y_test, pi.item())
#             X_test = torch.cat([Y_test, Pi_test], dim=1)

#             U_pred_dimless = (best_model(X_test).cpu().numpy().flatten())
#             U_analytical_dimless = (pi.item()/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

#             mse = np.mean((U_pred_dimless - U_analytical_dimless)**2)
#             total_loss += mse

#     avg_loss = total_loss / min(5, len(pi_values))
#     print(f"{dataset_name} set average MSE: {avg_loss:.2e}")

# evaluate_dataset(dm.pi_train, "Train")
# evaluate_dataset(dm.pi_val, "Validation")
# evaluate_dataset(dm.pi_test, "Test")

In [ ]:
task.close()

In [ ]:
# ----------------------------
# Testing and Results
# ----------------------------
best_model = PINN.load_from_checkpoint(ckpt_cb.best_model_path, layers=cfg.layers, lr=cfg.lr)
best_model.eval()
best_model = best_model.to(device)

# Test on various Pi values
test_pi_values = [0.5, 1.0, 3.0, 5.0, 7.0, 10.0]
Y_test = torch.linspace(-0.5, 0.5, 100, device=device).reshape(-1, 1)

# Results tables
dimensionless_results = []
dimensional_results = []
detailed_results = []

# Calculate U_ref for each Pi and Reynolds number
def calculate_reynolds(pi, cfg):
    """Calculate Reynolds number for given Pi"""
    # From Pi definition: Π = (h²/(μ·U_ref)) * (dp/dx)
    # So U_ref = (h² * |dp/dx|) / (μ * Π)  [taking absolute value for Re calculation]
    U_ref = (cfg.h**2 * cfg.dpdx) / (cfg.mu * pi)
    Re = (cfg.rho * abs(U_ref) * cfg.h) / cfg.mu
    return Re, U_ref

# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
colors = plt.cm.viridis(np.linspace(0, 1, len(test_pi_values)))

# Plot 1: Dimensionless velocity profiles
ax1 = axes[0, 0]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

    # Solid line for PINN, crosses for analytical
    ax1.plot(Y_test.cpu().numpy(), U_pred_dimless, '-', color=colors[i], linewidth=2, label=f'PINN Π={pi}')
    ax1.plot(Y_test.cpu().numpy(), U_analytical_dimless, 'x', color=colors[i], markersize=4, label=f'Analytical Π={pi}')

ax1.set_xlabel('Y (Dimensionless)')
ax1.set_ylabel('U (Dimensionless)')
ax1.set_title('Dimensionless Velocity Profiles')
ax1.legend()
ax1.grid(True)

# Plot 2: Dimensional velocity profiles
ax2 = axes[0, 1]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    # Calculate U_ref for this Pi
    Re, U_ref = calculate_reynolds(pi, cfg)

    # Convert to dimensional
    u_pred_dimensional = U_pred_dimless * U_ref
    y_dimensional = Y_test.cpu().numpy().flatten() * cfg.h

    # Analytical solution in dimensional form
    u_analytical_dimensional = analytical_solution_dimensional(
        y_dimensional, cfg.dpdx, cfg.mu, cfg.h
    )

    ax2.plot(y_dimensional, u_pred_dimensional, '-', color=colors[i], linewidth=2, label=f'PINN Π={pi}')
    ax2.plot(y_dimensional, u_analytical_dimensional, 'x', color=colors[i], markersize=4, label=f'Analytical Π={pi}')

ax2.set_xlabel('y [m]')
ax2.set_ylabel('u [m/s]')
ax2.set_title('Dimensional Velocity Profiles')
ax2.legend()
ax2.grid(True)

# Plot 3: Absolute velocity error (dimensionless)
ax3 = axes[1, 0]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

    absolute_error = np.abs(U_pred_dimless - U_analytical_dimless)

    ax3.plot(Y_test.cpu().numpy(), absolute_error, '-', color=colors[i], linewidth=2, label=f'Π={pi}')

ax3.set_xlabel('Y (Dimensionless)')
ax3.set_ylabel('Absolute Error (Dimensionless)')
ax3.set_title('Absolute Velocity Error (Dimensionless)')
ax3.legend()
ax3.grid(True)

# Plot 4: Error distribution across Pi values
ax4 = axes[1, 1]
max_errors = []
center_errors = []
for pi in test_pi_values:
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

    max_error = np.max(np.abs(U_pred_dimless - U_analytical_dimless))
    center_error = np.abs(U_pred_dimless[50] - U_analytical_dimless[50])

    max_errors.append(max_error)
    center_errors.append(center_error)

ax4.plot(test_pi_values, max_errors, 'o-', linewidth=2, label='Max Error')
ax4.plot(test_pi_values, center_errors, 's-', linewidth=2, label='Center Error')
ax4.set_xlabel('Π')
ax4.set_ylabel('Error (Dimensionless)')
ax4.set_title('Error vs Π')
ax4.legend()
ax4.grid(True)
ax4.set_yscale('log')

plt.tight_layout()
plt.show()

# Calculate detailed results with Reynolds numbers
print("\n" + "="*80)
print("COMPREHENSIVE TEST RESULTS")
print("="*80)

for pi in test_pi_values:
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

    # Calculate Reynolds number and U_ref
    Re, U_ref = calculate_reynolds(pi, cfg)

    # Convert to dimensional
    u_pred_dimensional = U_pred_dimless * U_ref
    u_analytical_dimensional = U_analytical_dimless * U_ref
    y_dimensional = Y_test.cpu().numpy().flatten() * cfg.h

    # Errors
    mse_dimless = np.mean((U_pred_dimless - U_analytical_dimless)**2)
    max_error_dimless = np.max(np.abs(U_pred_dimless - U_analytical_dimless))

    mse_dimensional = np.mean((u_pred_dimensional - u_analytical_dimensional)**2)
    max_error_dimensional = np.max(np.abs(u_pred_dimensional - u_analytical_dimensional))

    # Maximum velocity (at center)
    max_u_pred = U_pred_dimless[50]  # Y=0
    max_u_analytical = U_analytical_dimless[50]
    max_u_error = abs(max_u_pred - max_u_analytical)

    max_u_pred_dimensional = u_pred_dimensional[50]
    max_u_analytical_dimensional = u_analytical_dimensional[50]
    max_u_error_dimensional = abs(max_u_pred_dimensional - max_u_analytical_dimensional)

    dimensionless_results.append({
        "Pi": pi,
        "MSE": mse_dimless,
        "Max_Error": max_error_dimless,
        "U_pred_center": max_u_pred,
        "U_analytical_center": max_u_analytical,
        "Error_center": max_u_error
    })

    dimensional_results.append({
        "Pi": pi,
        "U_ref [m/s]": U_ref,
        "Re": Re,
        "MSE [m²/s²]": mse_dimensional,
        "Max_Error [m/s]": max_error_dimensional,
        "u_pred_center [m/s]": max_u_pred_dimensional,
        "u_analytical_center [m/s]": max_u_analytical_dimensional,
        "Error_center [m/s]": max_u_error_dimensional
    })

    detailed_results.append({
        "#": len(detailed_results) + 1,
        "Pi": pi,
        "Re": f"{Re:.2f}",
        "Max_U_Pred": f"{max_u_pred:.6f}",
        "Max_U_Analytical": f"{max_u_analytical:.6f}",
        "Max_U_Error": f"{max_u_error:.6f}",
        "MSE": f"{mse_dimless:.6f}"
    })

# Print detailed table with Re numbers
print("\nDETAILED DIMENSIONLESS RESULTS WITH REYNOLDS NUMBERS")
print("="*80)
detailed_df = pd.DataFrame(detailed_results)
print(detailed_df.to_string(index=False))

# Print dimensionless results
print("\n" + "="*80)
print("DIMENSIONLESS RESULTS")
print("="*80)
df_dimensionless = pd.DataFrame(dimensionless_results)
pd.set_option("display.float_format", "{:.6e}".format)
print(df_dimensionless.to_string(index=False))

# Print dimensional results
print("\n" + "="*80)
print("DIMENSIONAL RESULTS")
print("="*80)
df_dimensional = pd.DataFrame(dimensional_results)
print(df_dimensional.to_string(index=False))

# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"Training Pi range: {cfg.pi_min} to {cfg.pi_max}")
print(f"Test Pi values: {test_pi_values}")
print(f"Physical parameters: h={cfg.h} m, μ={cfg.mu} Pa·s, ρ={cfg.rho} kg/m³, dp/dx={cfg.dpdx} Pa/m")
print(f"\nDimensionless Max Error across all tests: {max(r['Max_Error'] for r in dimensionless_results):.2e}")
print(f"Dimensional Max Error across all tests: {max(r['Max_Error [m/s]'] for r in dimensional_results):.2e} m/s")

# Reynolds number analysis
print(f"\nReynolds Number Range: {min(float(r['Re']) for r in dimensional_results):.2f} to {max(float(r['Re']) for r in dimensional_results):.2f}")

# Test on validation and test sets
print("\n" + "="*80)
print("VALIDATION AND TEST SET PERFORMANCE")
print("="*80)

def evaluate_dataset(pi_values, dataset_name):
    total_mse = 0
    total_max_error = 0
    count = 0

    with torch.no_grad():
        for pi in pi_values[:10]:  # Evaluate on first 10 values
            Pi_test = torch.full_like(Y_test, pi.item())
            X_test = torch.cat([Y_test, Pi_test], dim=1)

            U_pred_dimless = best_model(X_test).cpu().numpy().flatten()
            U_analytical_dimless = (pi.item()/2.0) * (Y_test.cpu().numpy().flatten()**2 - 0.25)

            mse = np.mean((U_pred_dimless - U_analytical_dimless)**2)
            max_error = np.max(np.abs(U_pred_dimless - U_analytical_dimless))

            total_mse += mse
            total_max_error += max_error
            count += 1

    avg_mse = total_mse / count
    avg_max_error = total_max_error / count
    print(f"{dataset_name} set - Avg MSE: {avg_mse:.2e}, Avg Max Error: {avg_max_error:.2e}")

evaluate_dataset(dm.pi_train, "Train")
evaluate_dataset(dm.pi_val, "Validation")
evaluate_dataset(dm.pi_test, "Test")

# Additional analysis: Check if the model learned the physical relationship
print("\n" + "="*80)
print("PHYSICAL RELATIONSHIP ANALYSIS")
print("="*80)
print("Checking if U_max ∝ Π (should be linear relationship):")

# Extract max velocities vs Pi
pi_values_analysis = np.array([r['Pi'] for r in dimensionless_results])
max_u_values = np.array([r['U_pred_center'] for r in dimensionless_results])
max_u_analytical = np.array([r['U_analytical_center'] for r in dimensionless_results])

# Calculate linear regression for predicted values
slope_pred, intercept_pred = np.polyfit(pi_values_analysis, max_u_values, 1)
slope_analytical, intercept_analytical = np.polyfit(pi_values_analysis, max_u_analytical, 1)

print(f"Predicted relationship: U_max = {slope_pred:.6f} * Π + {intercept_pred:.6f}")
print(f"Analytical relationship: U_max = {slope_analytical:.6f} * Π + {intercept_analytical:.6f}")
print(f"Slope error: {abs(slope_pred - slope_analytical)/slope_analytical*100:.2f}%")

# Plot the relationship
plt.figure(figsize=(10, 6))
plt.plot(pi_values_analysis, max_u_values, 'bo-', label='PINN Prediction', linewidth=2)
plt.plot(pi_values_analysis, max_u_analytical, 'rx-', label='Analytical', linewidth=2)
plt.plot(pi_values_analysis, slope_pred * pi_values_analysis + intercept_pred, 'b--', alpha=0.5, label=f'PINN Fit: slope={slope_pred:.4f}')
plt.plot(pi_values_analysis, slope_analytical * pi_values_analysis + intercept_analytical, 'r--', alpha=0.5, label=f'Analytical: slope={slope_analytical:.4f}')
plt.xlabel('Π')
plt.ylabel('U_max (at Y=0)')
plt.title('Maximum Velocity vs Π (Checking Physical Relationship)')
plt.legend()
plt.grid(True)
plt.show()